# Detectron2 Beginner's Tutorial

<img src="https://dl.fbaipublicfiles.com/detectron2/Detectron2-Logo-Horz.png" width="500">

Welcome to detectron2! This is the official colab tutorial of detectron2. Here, we will go through some basics usage of detectron2, including the following:
* Run inference on images or videos, with an existing detectron2 model
* Train a detectron2 model on a new dataset

You can make a copy of this tutorial by "File -> Open in playground mode" and make changes there. __DO NOT__ request access to this tutorial.


# Install detectron2

In [1]:
!nvidia-smi
!nvcc -V

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


### Update and purge Nvidia Driver
#### https://medium.com/analytics-vidhya/install-cuda-11-2-cudnn-8-1-0-and-python-3-9-on-rtx3090-for-deep-learning-fcf96c95f7a1

In [2]:
!python -c "import torch; print(torch.__version__); import torchvision; print(torchvision.__version__)"

1.8.1+cu111
0.9.1+cu111


In [3]:
# install dependencies: 
!pip install pyyaml==5.1
!pip install cmake
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

1.8.1+cu111 False
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [4]:
# Manually install torch 1.8.1 as bug in 1.8.0
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio==0.8.1 -f https://download.pytorch.org/whl/torch_stable.html
#!pip install torch torchvision torchaudio -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
!pip install torchtext==0.9.1

#exit(0) # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [5]:
print(torch.__version__, torch.cuda.is_available())

1.8.1+cu111 False


In [6]:
# install detectron2: (Colab has CUDA 10.1 + torch 1.8)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
import torch
assert torch.__version__.startswith("1.8.")   # need to manually install torch 1.8 if Colab changes its default version
!python -m pip install detectron2==0.4 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
#exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
     |████████████████████████████████| 6.2MB 516kB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
     |████████████████████████████████| 81kB 9.2MB/s 
     |████████████████████████████████| 112kB 43.8MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.3.post20210317-cp37-none-any.whl size=58543 sha256=a3da656e642d8222ea2bd1b3deed822f55153e64661d306476216e515fbfae0e
  Stored in directory: /root/.cache/pip/wheels/d2/ee/3a/5c531df777c03d8c67f22c65f97d6f75321087482d05a9b218
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-cp37-none-any.whl size=141231 sha256=43c8ed3b803be70a30e10e98a9f8bc8967150a1dfce6c60d4692aba237cd14dc
  Stored in directory: /root/.cache/pip/wheels/e3/e2/fa/b78480b448b8579ddf393bebd3f47ee23aa84c89b6a78285c8
Successfully built fvcore antlr4-python3-runtime


In [7]:
!python -c 'import torch;print(torch.eye(3))'
!python -c 'from torch.utils.collect_env import main; main()'

tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])
PyTorch version: 1.8.1+cu111
Is debug build: False
CUDA used to build PyTorch: 11.1
ROCM used to build PyTorch: N/A

OS: Ubuntu 18.04.5 LTS (x86_64)
GCC version: (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Clang version: 6.0.0-1ubuntu2 (tags/RELEASE_600/final)
CMake version: version 3.12.0

Python version: 3.7 (64-bit runtime)
Is CUDA available: False
CUDA runtime version: No CUDA
GPU models and configuration: No CUDA
Nvidia driver version: No CUDA
cuDNN version: No CUDA
HIP runtime version: N/A
MIOpen runtime version: N/A

Versions of relevant libraries:
[pip3] numpy==1.19.5
[pip3] torch==1.8.1+cu111
[pip3] torchaudio==0.8.1
[pip3] torchsummary==1.5.1
[pip3] torchtext==0.9.1
[pip3] torchvision==0.9.1+cu111
[conda] Could not collect


In [14]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

## FineTuning PubLayNet

In [15]:
# download, decompress the data
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1NAgGrYoQJwdNXE-nFzx0yEJEoqxMcqh2' -O siemens.zip # exchange
!unzip siemens.zip -d datasets
!rm -r siemens.zip
!ls 'datasets/siemens'

--2021-06-25 10:08:32--  https://docs.google.com/uc?export=download&id=1NAgGrYoQJwdNXE-nFzx0yEJEoqxMcqh2
Resolving docs.google.com (docs.google.com)... 172.217.13.78, 2607:f8b0:4004:829::200e
Connecting to docs.google.com (docs.google.com)|172.217.13.78|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-04-9c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ruti4s3m00snb0vj9s46amd1gs2fm711/1624615650000/11785652857695517745/*/1NAgGrYoQJwdNXE-nFzx0yEJEoqxMcqh2?e=download [following]
--2021-06-25 10:08:33--  https://doc-04-9c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ruti4s3m00snb0vj9s46amd1gs2fm711/1624615650000/11785652857695517745/*/1NAgGrYoQJwdNXE-nFzx0yEJEoqxMcqh2?e=download
Resolving doc-04-9c-docs.googleusercontent.com (doc-04-9c-docs.googleusercontent.com)... 142.251.33.193, 2607:f8b0:4004:837::2001
Connecting to doc-04-9c-docs.googleusercontent.com (doc-04-9c-docs.googleu

### Register Custom Dataset

In [10]:
from detectron2.data.datasets import load_coco_json, register_coco_instances
from detectron2.data import MetadataCatalog

cur_dir = os.getcwd()
data_dir = os.path.join(cur_dir, "datasets/siemens")

# Training dataset
training_dataset_name = "dla_train"
training_json_file = os.path.join(data_dir, "train/train2021.json")
training_img_dir = os.path.join(data_dir, "train/images")

# Register custom training dataset
register_coco_instances(training_dataset_name, {}, training_json_file, training_img_dir)

# Validation dataset
validation_dataset_name = "dla_val"
validation_json_file = os.path.join(data_dir, "val/val2021.json")
validation_img_dir = os.path.join(data_dir, "val/images")
    
# Register custom training dataset
register_coco_instances(validation_dataset_name, {}, validation_json_file, validation_img_dir)
    
# Get training/validation dictionary
training_dict = load_coco_json(training_json_file, training_img_dir, dataset_name=training_dataset_name)
validation_dict = load_coco_json(validation_json_file, validation_img_dir, dataset_name=validation_dataset_name)

# Register metadata
training_metadata = MetadataCatalog.get(training_dataset_name)
validation_metadata = MetadataCatalog.get(validation_dataset_name)

WARNING [06/25 10:07:48 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[06/25 10:07:48 d2.data.datasets.coco]: Loaded 6 images in COCO format from /content/datasets/siemens/train/train2021.json
WARNING [06/25 10:07:48 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[06/25 10:07:48 d2.data.datasets.coco]: Loaded 2 images in COCO format from /content/datasets/siemens/val/val2021.json


In [11]:
!git clone https://github.com/JaninaMattes/detectron2.git

Cloning into 'detectron2'...
remote: Enumerating objects: 2687, done.
remote: Counting objects: 100% (185/185), done.
remote: Compressing objects: 100% (155/155), done.
remote: Total 2687 (delta 54), reused 98 (delta 29), pack-reused 2502
Receiving objects: 100% (2687/2687), 37.82 MiB | 9.56 MiB/s, done.
Resolving deltas: 100% (1691/1691), done.


### Get pre-trained model

In [12]:
# download pre-trained model
!wget https://www.dropbox.com/sh/wgt9skz67usliei/AADGw0h1y7K5vO0akulyXm-qa/model_final.pth
!ls

--2021-06-25 10:07:58--  https://www.dropbox.com/sh/wgt9skz67usliei/AADGw0h1y7K5vO0akulyXm-qa/model_final.pth
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/wgt9skz67usliei/AADGw0h1y7K5vO0akulyXm-qa/model_final.pth [following]
--2021-06-25 10:07:58--  https://www.dropbox.com/sh/raw/wgt9skz67usliei/AADGw0h1y7K5vO0akulyXm-qa/model_final.pth
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc7aa72c9744e585564d7c6847f.dl.dropboxusercontent.com/cd/0/inline/BRFGSlXzXe_zdIE25Z47qz7xRUOV_JeSpW4DyrrFNY_7j7qNBdR0ez3SbdZoCeqOPVq9LNoMUh-kUZGg5RQI0gXB2UffaLgeB59VazdUpzVqBppWwYEWXBh-aPIhzomZQYb5lU10Ux8qhH9gZdcxWT4I/file# [following]
--2021-06-25 10:07:58--  https://ucc7aa72c9744e585564d7c6847f.dl.dropboxusercontent.com/cd/0/inline/

### Train the pre-trained Mask-RCNN R50 FPN Model

In [17]:
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer, default_argument_parser, default_setup, hooks, launch

cfg = get_cfg()
cfg.merge_from_file("detectron2/configs/DLA_mask_rcnn_R_101_FPN_3x.yaml")
cfg.DATASETS.TRAIN = ("dla_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 1000
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4

# Create output
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

# Serialize the training config
cfg_str = cfg.dump()
with open(os.path.join(cfg.OUTPUT_DIR, "train_config.yaml"), "w") as f:
    f.write(cfg_str)
f.close()

# Start training with Default Trainer
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

WARNING [06/25 10:10:21 d2.config.compat]: Config 'detectron2/configs/DLA_mask_rcnn_R_101_FPN_3x.yaml' has no VERSION. Assuming it to be compatible with latest v2.


RuntimeError: ignored

### Tensorboard

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

### Evaluation of model training

In [ ]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [ ]:
from detectron2.utils.visualizer import ColorMode
dataset_dicts = get_balloon_dicts("siemens_dataset/val")
for d in random.sample(dataset_dicts, 3):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    v = Visualizer(im[:, :, ::-1],
                   metadata=balloon_metadata, 
                   scale=0.5, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])

### Check metrics for COCO API 

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("dla_val", ("bbox", "segm"), False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "dla_val")
print(inference_on_dataset(trainer.model, val_loader, evaluator))
# another equivalent way to evaluate the model is to use `trainer.test`

### Download fine-tuned model

In [ ]:
files.download('fine_tunes_model.pth')